In [3]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')  # For pos_tag
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/msawant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/msawant/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/msawant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/msawant/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/msawant/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix,f1_score, precision_score, recall_score

In [6]:
import pandas as pd
#file_path = "EXIST_2023/disagreement_records_task1.xlsx"

#file_path_f="/content/drive/MyDrive/Experiment2/age1_disagreement_records.xlsx"
#file_path_a2="EXIST_2023/age2_disagreement_records.xlsx"
#file_path_a3="EXIST_2023/age3_disagreement_records.xlsx"
file_path_f="EXIST_2023/female_disagreement_records.xlsx"
#file_path_m="EXIST_2023/male_disagreement_records.xlsx"

#file_path_f="/content/drive/MyDrive/Experiment2/female_disagreement_records.xlsx"
#file_path_f="/content/drive/MyDrive/Experiment2/male_disagreement_records.xlsx"

#file_path_f="/content/drive/MyDrive/Experiment2/age2_disagreement_records.xlsx"
#file_path_f="/content/drive/MyDrive/Experiment2/age3_disagreement_records.xlsx"
#file_path = "EXIST_2023/EXIST2023_Task1_en.xlsx"
#data_a1 = pd.read_excel(file_path_a1)
#data_a3 = pd.read_excel(file_path_a3)
#data_a2 = pd.read_excel(file_path_a2)
data_f = pd.read_excel(file_path_f)
#data_m = pd.read_excel(file_path_m)

In [7]:
import re

def remove_urls_and_lower(text):
    # Define the regex pattern for URLs starting with http or https
    url_pattern = re.compile(r'http[s]?://\S+')
    # Substitute the URLs with an empty string
    cleaned_text = url_pattern.sub('', text)
    cleaned_text = cleaned_text.lower()
    return cleaned_text.strip()

In [8]:
def clean_text(text):
    return re.sub(r'[^A-Za-z\s]', '', text) if isinstance(text, str) else text
    #return re.sub(r'[^A-Za-z0-9\s]', '', text) if isinstance(text, str) else text

In [9]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
#additional_stopwords={'ago','also','along','always','amp','like','look','know','even','want','say','get','far','use','take','never','great','whole','know','even','use','take','would'}
stop_words.update(['ago','also','along','always','amp','like','look','know','even','want','say','get','far','use','take','never','great','whole','would','ok','dont','cant','shes','theyre','without','youre','isnt','us','yet','yall','u','id'])
#stop_words.update(['ago','also','along','always','amp','like','look','know','even','want','say','get','far','use','take','never','great','whole','would','ok','dont','cant','shes','theyre','without','youre','id','call','make'])
def stopwords_removal(text):
    words=word_tokenize(text)
    filtered_words=[w for w in words if w.lower() not in stop_words]
    return ' '.join(filtered_words)

In [10]:
from nltk.corpus import wordnet
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [11]:
from nltk.stem import WordNetLemmatizer
def apply_lemmatization(text):
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(text))
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:
        # else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    lemmatized_sentence = " ".join(lemmatized_sentence)
    return lemmatized_sentence

Female subset of disagreement

In [12]:
tweet_processed_f=data_f["tweet"].apply(remove_urls_and_lower)
tweet_processed_f=tweet_processed_f.apply(clean_text)
tweet_processed_f=tweet_processed_f.apply(apply_lemmatization)
tweet_processed_f=tweet_processed_f.apply(stopwords_removal)

In [13]:
vec_f=TfidfVectorizer(tokenizer=word_tokenize,token_pattern=None,ngram_range=(1,1))
tweet_vectorized_f=vec_f.fit_transform(tweet_processed_f)

In [14]:
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(tweet_vectorized_f,data_f["majority_vote"], test_size=0.30, random_state=0)

In [15]:
rf_model_f=RandomForestClassifier(bootstrap=False, min_samples_leaf=4,min_samples_split=10,n_estimators=100,n_jobs=-1,random_state=10)
rf_model_f.fit(X_train_f, y_train_f)

RandomForestClassifier(bootstrap=False, min_samples_leaf=4,
                       min_samples_split=10, n_jobs=-1, random_state=10)

In [16]:
y_pred_rf_f =rf_model_f.predict(X_test_f)
y_pred_rf_f = y_pred_rf_f.astype(int)
accuracy= accuracy_score(y_test_f,y_pred_rf_f)
print("Female Disagreement subset")
print("ACCURACY OF THE MODEL:", metrics.accuracy_score(y_test_f, y_pred_rf_f))
f1=f1_score(y_test_f,y_pred_rf_f,average='macro')
print(f"F1 score:{f1}")

# Calculate Precision and Recall
precision = precision_score(y_test_f, y_pred_rf_f, average='macro')
recall = recall_score(y_test_f, y_pred_rf_f, average='macro')

print(f'Precision: {precision}')
print(f'Recall: {recall}')

Female Disagreement subset
ACCURACY OF THE MODEL: 0.5935334872979214
F1 score:0.538938095929529
Precision: 0.5824369436201781
Recall: 0.5579027355623101


In [17]:
target_names = ['Yes','No']
print(classification_report(y_test_f, y_pred_rf_f, target_names=target_names))

              precision    recall  f1-score   support

         Yes       0.56      0.29      0.38       188
          No       0.60      0.83      0.70       245

    accuracy                           0.59       433
   macro avg       0.58      0.56      0.54       433
weighted avg       0.59      0.59      0.56       433



SHAP

In [18]:
from sklearn.pipeline import make_pipeline
c = make_pipeline(vec_f, rf_model_f)
clf_predict = c.predict
clf_predict_proba = c.predict_proba

In [19]:

x=int(input("Enter the index for explanation: "))
text_instance = data_f["tweet"][x]
#text_instance=remove_urls_and_lower(text_instance)
#text_instance=apply_lemmatization(text_instance)
text_instance=clean_text(text_instance)
#text_instance=stopwords_removal(text_instance)
print(text_instance)
#For age1 used id 300
#For female used id 250
#For male used id 350
#For age2 used id 400
#For age3 used id 200

NutDivide A year of cock tease over how full it is Well played sir well played I do wonder though at this rate if Biden does  years are you gonna need a bigger jar


In [20]:
print(text_instance)

NutDivide A year of cock tease over how full it is Well played sir well played I do wonder though at this rate if Biden does  years are you gonna need a bigger jar


In [21]:
value=data_f["majority_vote"][x]
print(value)

0


In [22]:
import shap

/Users/msawant/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
class_names=[1,0]

In [24]:
masker = shap.maskers.Text(tokenizer=r"\W+")
explainer = shap.Explainer(clf_predict_proba, masker=masker, output_names=class_names)
shap_values = explainer([text_instance])

In [27]:
shap.plots.text(shap_values[:])